# Cap Tag Reports

## Import Modules

In [36]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
import selenium.webdriver as webdriver
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import json
import cx_Oracle
import os

opts = Options()
opts.headless = True
opts.add_argument('--ignore-certificate-errors')
opts.add_argument('--start-maximized')
prefs ={"profile.default_content_settings.popups": 0, "download.default_directory": "C:\\Users\\wb5888\\Python Code\\CapReport\\Downloads\\", "directory_upgrade": True}
opts.add_experimental_option("prefs", prefs)
assert opts.headless
browser = Chrome(executable_path = 'C:\\Users\\wb5888\\chromedriver.exe', options = opts)

os.getcwd()

'C:\\Users\\wb5888\\Documents\\Python Code\\CapReport'

## Run Cap Tag Report

Enter PR number, revision number and directory to output Cap Tag Report.

Can change database if needed.

In [3]:
pr_num = "1-IWH6ME"
rev_num = "2"
pe_db = "TPPE"

write_dir = 'C:\\Users\\wb5888\\Documents\\Cap Tags'

## Set SQL Commands

### Cap Report

In [4]:
selectstr1 = "(select distinct F.name as CustomerName, F.Customerid, B.name as LDC_Account, B.Accountid,"
selectstr1 = "".join([selectstr1, "D.uidaccount, D.marketcode, A.Contractid, A.Revision "])
selectstr1 = "".join([selectstr1, "from pwrline.account B, pwrline.lscmcontract A, pwrline.lscmcontractitem C, "])
selectstr1 = "".join([selectstr1, "pwrline.acctservicehist D, pwrline.customer F "])
selectstr1 = "".join([selectstr1, "where C.uidcontract=A.uidcontract and C.uidaccount=B.uidaccount and B.uidaccount=D.uidaccount "])
selectstr1 = "".join([selectstr1, "and B.uidcustomer=F.uidcustomer and A.contractid='", pr_num, "' and A.revision=", rev_num, ") A "])

s2str = "select distinct A.*, B.starttime, B.stoptime, B.overridecode as Tag_Type, B.val as Tag, B.strval as SOURCE_TYPE, B.lstime as Timestamp "
s2str = "".join([s2str, "from pwrline.acctoverridehist B, ", selectstr1])
s2str = "".join([s2str, "where A.uidaccount=B.uidaccount and (A.marketcode='PJM' OR  A.marketcode='NEPOOL' OR A.marketcode= 'NYISO' OR A.marketcode= 'MISO') "])
s2str = "".join([s2str, "and (B.overridecode ='TRANSMISSION_TAG_OVRD' OR B.overridecode='CAPACITY_TAG_OVRD') "])
s2str = "".join([s2str, "order by A.customername, B.overridecode, A.accountid, B.starttime"])

uid = 'tesi_interface'
pwd = 'peint88'

ip = '172.25.152.125'
port = '1700'
service_name = 'tppe.mytna.com'
dsn = cx_Oracle.makedsn(ip, port, service_name=service_name)

print(s2str)

select distinct A.*, B.starttime, B.stoptime, B.overridecode as Tag_Type, B.val as Tag, B.strval as SOURCE_TYPE, B.lstime as Timestamp from pwrline.acctoverridehist B, (select distinct F.name as CustomerName, F.Customerid, B.name as LDC_Account, B.Accountid,D.uidaccount, D.marketcode, A.Contractid, A.Revision from pwrline.account B, pwrline.lscmcontract A, pwrline.lscmcontractitem C, pwrline.acctservicehist D, pwrline.customer F where C.uidcontract=A.uidcontract and C.uidaccount=B.uidaccount and B.uidaccount=D.uidaccount and B.uidcustomer=F.uidcustomer and A.contractid='1-IWH6ME' and A.revision=2) A where A.uidaccount=B.uidaccount and (A.marketcode='PJM' OR  A.marketcode='NEPOOL' OR A.marketcode= 'NYISO' OR A.marketcode= 'MISO') and (B.overridecode ='TRANSMISSION_TAG_OVRD' OR B.overridecode='CAPACITY_TAG_OVRD') order by A.customername, B.overridecode, A.accountid, B.starttime


### Accounts

## Define Functions

In [5]:
def OracleAPI(query):
    
    uid = 'tesi_interface'
    pwd = 'peint88'

    ip = '172.25.152.125'
    port = '1700'
    service_name = 'tppe.mytna.com'
    dsn = cx_Oracle.makedsn(ip, port, service_name=service_name)
    
    result_list = []
    con = cx_Oracle.connect(user = uid, password = pwd, dsn = dsn)
    cur = con.cursor()
    cur.execute(query)
    for result in cur:
        result_list.append(result)
    return(result_list)

In [6]:
output = OracleAPI(s2str)

captag = pd.DataFrame.from_records(output)
captag.columns = ['CustomerName', 'CustomerID', 'LDC_Account', 'AccountID', 'UIDaccount', 'MarketCode', 'ContractID', 'Revision', 'StartTime', 'StopTime', 'TagType', 'Tag', 'SourceType', 'TimeStamp']

captag

,CustomerName,CustomerID,LDC_Account,AccountID,UIDaccount,MarketCode,ContractID,Revision,StartTime,StopTime,TagType,Tag,SourceType,TimeStamp
0,"BOSCOV'S DEPARTMENT STORE, LLC",1-2I62Q,1471983017,NEPOOL_NRI_1471983017,1527853,NEPOOL,1-IWH6ME,2,2018-06-01,2019-05-31 23:59:59,CAPACITY_TAG_OVRD,380.53,H,2019-05-08 15:48:08
1,"BOSCOV'S DEPARTMENT STORE, LLC",1-2I62Q,1471983017,NEPOOL_NRI_1471983017,1527853,NEPOOL,1-IWH6ME,2,2019-06-01,2020-05-31 23:59:59,CAPACITY_TAG_OVRD,380.53,S,2019-05-08 15:48:08
2,"BOSCOV'S DEPARTMENT STORE, LLC",1-2I62Q,1471983017,NEPOOL_NRI_1471983017,1527853,NEPOOL,1-IWH6ME,2,2020-06-01,2030-05-31 23:59:59,CAPACITY_TAG_OVRD,380.53,S,2019-05-08 15:48:08
3,"BOSCOV'S DEPARTMENT STORE, LLC",1-2I62Q,6455634014,NEPOOL_NRI_6455634014,1528118,NEPOOL,1-IWH6ME,2,2018-06-01,2019-05-31 23:59:59,CAPACITY_TAG_OVRD,256.10,E,2019-05-09 10:28:26
4,"BOSCOV'S DEPARTMENT STORE, LLC",1-2I62Q,6455634014,NEPOOL_NRI_6455634014,1528118,NEPOOL,1-IWH6ME,2,2019-06-01,2020-05-31 23:59:59,CAPACITY_TAG_OVRD,256.10,S,2019-05-09 10:28:26
5,"BOSCOV'S DEPARTMENT STORE, LLC",1-2I62Q,6455634014,NEPOOL_NRI_6455634014,1528118,NEPOOL,1-IWH6ME,2,2020-06-01,2030-05-31 23:59:59,CAPACITY_TAG_OVRD,256.10,S,2019-05-09 10:28:26


In [110]:
acct_id[0].split('NEPOOL_')[1]

'MECO_5297968001'

In [35]:
query = "select distinct * from pwrline.lscmcontract where contractid='" + pr_num + "' and revision=" + rev_num

PR_start = OracleAPI(query)[0][4]
PR_end = OracleAPI(query)[0][5]

print('PR goes from,', PR_start, 'to', PR_end)

PR goes from, 2019-06-01 00:00:00 to 2025-12-31 23:59:59


In [70]:
len(data)

60

In [39]:
acct = '27126606'
query = "select distinct B.AccountID from pwrline.acctservicehist D, pwrline.account B  where B.name like '%" + acct + "%' and D.marketcode = 'NEPOOL'"
OracleAPI(query)[0][0].split('NEPOOL_')[1]

'CLP_51271266066_097681001'

## EWX Scrape

In [38]:
browser.get('https://console.cloud.energyworx.com/')
browser.find_element_by_id